In [3]:
'''
No speecch from https://github.com/huggingface/transformers/issues/29595

We found that the proba-
bility of the <|nospeech|> token alone is not sufficient
to distinguish a segment with no speech, but combining
the no-speech probability threshold of 0.6 and the average
log-probability threshold of −1 makes the voice activity
detection of Whisper more reliable.

'''

import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import numpy as np
import pprint

# Load the processor and model
model_name = "vinai/PhoWhisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name).to("cuda")


/home/anh/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
def remove_possible_overlaps(transcriptions, maximum_overlapping_tokens = 5):
    cleaned_transcriptions = []

    for i, current in enumerate(transcriptions):
        if i == 0:
            # Add the first segment without changes
            cleaned_transcriptions.append(current)
            continue
        

        previous_text = cleaned_transcriptions[-1]['text'].split(' ')
        previous_text[-1] = previous_text[-1][:-1] if previous_text[-1][-1] == '.' else previous_text[-1]
        current_text  = current['text'].split(' ')

        for j in range(8):
            prev_tokens = previous_text[-j-1:]
            cur_tokens  = current_text[:j+1]
            if prev_tokens == cur_tokens:
                # Matched
                current_text = current_text[j+1:]
                break

        current['text'] = ' '.join(current_text)
        cleaned_transcriptions.append(current)

    return cleaned_transcriptions


def chunk_audio(audio_path, sample_rate=16000, segment_length=30, overlap = 1.):
    """Load and split audio into 30-second chunks, 0.5 seconds overlap."""
    audio, _ = librosa.load(audio_path, sr=sample_rate, mono=True)

    # Calculate the number of samples per segment and the overlap in samples
    num_samples = segment_length * sample_rate
    overlap_samples = int(overlap * sample_rate)

    # Create the chunks with overlap
    chunks = [
        audio[i:i + num_samples] 
        for i in range(0, len(audio) - overlap_samples, num_samples - overlap_samples)
    ]

    chunk_timestamps = [(i/sample_rate, (i + num_samples)/ float(sample_rate)) for i in range(0, len(audio) - overlap_samples, num_samples - overlap_samples)]

    return chunks, sample_rate, chunk_timestamps

def transcribe_chunk(chunk, processor, model, no_speech_threshold=0.6, logprob_threshold=-1.0):
    input_features = processor(chunk, sampling_rate=16000, return_tensors="pt").input_features.to("cuda")

    # Generate transcription with VAD filtering
    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            output_scores=True,
            return_dict_in_generate=True,
            max_new_tokens=400,
            no_speech_threshold=no_speech_threshold,
            logprob_threshold=logprob_threshold,
            temperature= (0.4 - 0.7)
        )
    
    # Decode and collect no-speech probability and log-probability information
    decoded_text = processor.batch_decode(predicted_ids.sequences, skip_special_tokens=True)[0]
    return {
        "text": decoded_text
    }


def transcribe_full(audio_path, processor, model, segment_length=30, no_speech_threshold=0.6, logprob_threshold=-1.0):
    # Chunk the audio
    chunks, _, timestamps = chunk_audio(audio_path, segment_length=segment_length)

    all_segments = []
    for index, chunk in enumerate(chunks):
        # Transcribe each chunk and gather segment details
        segment = transcribe_chunk(chunk, processor, model, no_speech_threshold, logprob_threshold)
        segment['timestamp'] = [timestamps[index][0], timestamps[index][1]]
        all_segments.append(segment)

    all_segments = remove_possible_overlaps(all_segments)

    return all_segments

In [56]:
# Example usage
audio_path = "sample/0 Giờ 2 Phút.mp3"
result = transcribe_full(audio_path, processor, model)

# Print the final filtered transcription
print("Transcribed Text:")
pprint.pprint(result)

/home/anh/.local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:777: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_scores` is. When `return_dict_in_generate` is not `True`, `output_scores` is ignored.
  warnings.warn(


Transcribed Text:
[{'text': 'giọng hồng cao cả.', 'timestamp': [0.0, 30.0]},
 {'text': 'ngắt nhẹ nhàng thoải lan làm sao lòng ta luôn nặng chiều bàn tay '
          'ngày anh đã yếu giữ lấy sao lại buồn có phải tình cảm đáng nhớ sơ '
          'hành nói đi đừng ai ngờ tại sao hai người nhắn thắc lại nước mắt '
          'của tôi.',
  'timestamp': [29.0, 59.0]},
 {'text': 'tại sao hai người khước đi những tháng ngày hạnh phúc giờ trong tôi '
          'là bấm đêm đang phải cuối những nỗi đau lòng tôi thấy ân hận đôi mí '
          'sưng sững lúc không giờ hài phố đến bên tôi vui đùa rồi xem tôi chỉ '
          'là một kẻ dư thương trái tim tôi đã lầm khi yêu long em sắp ê nhìn.',
  'timestamp': [58.0, 88.0]},
 {'text': 'những người đau khổ điều thức cùng nước mắt ngón sốc trong thâm tâm '
          'này không thể ta nói yêu thương cho nhiều giờ đây anh hiểu ra bao '
          'nhiêu điều suốt tình anh dễ dàng để đến từng vài ai những người '
          'hạnh phúc đều thật yên giấc còn mố

In [50]:
# transcriptions = [
#     {'text': 'những người đau khổ điều thức cùng nước mắt ngon sốc trong thâm tâm '
#              'này không thể tạo nó yêu thương cho nhiều giờ đây anh hiểu ra bao '
#              'nhiêu điều suốt tên anh dễ dàng để đến từng vậy ai những người hạnh '
#              'phúc đều thật yên giấc còn mối riêng anh ơi là không chợt mắt.',
#      'timestamp': [87.0, 117.0]},
#     {'text': 'không chợt mắt à à tại sao hai người nhận thân ngày muộn mất của tôi.',
#      'timestamp': [116.0, 146.0]},
#     {'text': 'tại sao hai người cướp đi những tháng ngày hạnh phúc giờ trong tôi '
#              'là bướm em đang vây quanh những nỗi đau lòng tôi thấy ân hận đôi mi '
#              'dừng dừng lúc không giờ hạnh phúc đến đêm tôi vui tựa dồi sáng tôi '
#              'chỉ là một kẻ dữ thừa trái tim tôi đã lầm ý hiếu sanh em sát tiểu.',
#      'timestamp': [145.0, 175.0]},
# ]

# new_transcriptions = remove_possible_overlaps(transcriptions)
# pprint.pprint(new_transcriptions)
